## Random Forests

Random Forests 

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer, PolynomialFeatures, StandardScaler
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from sktime.transformations.series.summarize import WindowSummarizer

In [2]:
%run Function_scripts/functions_model.py
%run Function_scripts/functions_vis.py

In [3]:
pd.set_option("display.max_columns", None)

### Loading the dataset

In [4]:
d = pd.read_csv("data/train_df.csv", parse_dates=[0])
d_test = pd.read_csv("data/test_df.csv", parse_dates = [0])

In [5]:
d['date'] = pd.to_datetime(d['date'])
d_test['date'] = pd.to_datetime(d_test['date'])

## Modelling 
#### Selecting and defining features 

The following features were selected to be included in the baseline model:

* Item Category
* Store
* Weather 
    * Temperature
    * Precipitation
    * Sunshine duration
* Time variables:
    * Timestep (number of days since the first recorded sale)
    * Day of the year
    * Weekday
    * Week of the year 
    * Month
    * Year
* Window variables
    * Lagged features
    * Rolling averages
    * Rolling standard deviation
* Special events 
    * New Year's Eve
    * Halloween
    * Valentine's Day
* Public Space dummy variable
* Public holidays
* Street market dummy variable

In [6]:
date = ["date"]

numfeat =["days_back","temperature_2m_mean","sunshine_duration","precipitation_hours"]

catfeat = ["store_name","item_category", 'day', 'halloween', 'hol_pub', "hol_school", 'month', 'nye', 'public_space', 'street_market', 'valentines_day','week_year', 'weekday', 'year']

In [7]:
d2 = d[date + catfeat + numfeat + ["total_amount"]]
d_test2 = d_test[date + catfeat + numfeat + ["total_amount"]]

In [8]:
agg_columns = d2.columns.difference(['date', 'store_name', 'item_category'] + ["total_amount"])
agg_dict = {col: "first" for col in agg_columns}
agg_dict["total_amount"] = "sum"

d2 = d2.groupby(['date', 'store_name', 'item_category']).agg(agg_dict).reset_index().sort_values(by = "date", ascending = False).reset_index(drop = True)
d2["hol_pub"] = d2["hol_pub"].apply(np.int64)

d2 = d2.set_index(["store_name","item_category","date"]).sort_index()

d_test2 = d_test2.groupby(['date', 'store_name', 'item_category']).agg(agg_dict).reset_index().sort_values(by = "date", ascending = False).reset_index(drop = True)
d_test2["hol_pub"] = d_test2["hol_pub"].apply(np.int64)

d_test2 = d_test2.set_index(["store_name","item_category","date"]).sort_index()

In [9]:
kwargs = {"lag_feature": {
    "lag":[1,2,3],
    "mean": [[1,7], [1, 15], [1,30]],
    "std": [[1,4]]
    },
    "target_cols":["total_amount"]}

transformer = WindowSummarizer(**kwargs, n_jobs= -1)

In [10]:
d2wind = transformer.fit_transform(d2)
d2wind = pd.concat([d2["total_amount"], d2wind], axis = 1).dropna()

### Transforming features

In order for the linear regression model to work properly, some additional feature preprocessing is necessary. For Random Forests, this only includes one-hot-encoding categorical features.

In [11]:
cat_tr =Pipeline(steps=[
  ("ohe", OneHotEncoder(drop='first',sparse_output=False))
])

In [12]:
prepro =ColumnTransformer(
  transformers=[
    ("cat",cat_tr,catfeat)
    ],
    remainder ='passthrough'
    
    )

In [64]:
seed = 21

rf =Pipeline(
  steps=[
    ("prepro", prepro),
    ("rf",RandomForestRegressor(random_state = seed, n_jobs = -1, verbose = 1, criterion="poisson"))])

In [57]:
xtrain = d2wind.reset_index().drop("total_amount", axis = 1).set_index("date")
ytrain = d2wind.reset_index()[["date","total_amount"]].set_index("date")

### Model fit and prediction

**Time-series cross-validation**

When performing k-fold cross-validation in a time-series context, the data musn't be shuffled as in a regular cross-validation scenario. Instead, a forecasting horizon has to be defined for each fold, which serves as the validation set. It always comes after the training set chronologically. Because predicting donut sales for more than one week in advance is a difficult endeavor and will most likely lead to imprecise results, the forecasting horizon for this problem was set to 7 days.

For more information, please refer to the notebook with the Catboost model.

In [58]:
tscv = create_train_validation_folds(xtrain.reset_index())

In [71]:
param={
  "rf__max_depth":np.arange(5,50,1),
  "rf__n_estimators":np.arange(50,500,10),
  "rf__min_samples_split":np.arange(4,10,1),
  "rf__min_samples_leaf":np.arange(2,10,1),
  "rf__max_features": ["sqrt","log2"]
  }

rfh =RandomizedSearchCV(estimator= rf, param_distributions = param, refit = "neg_root_mean_squared_error", verbose = 1,
      scoring = ["neg_root_mean_squared_error","r2",'neg_mean_absolute_percentage_error'], cv=tscv, n_jobs = -1, n_iter = 20)


In [72]:
rfh.fit(xtrain, ytrain)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
pd.DataFrame(rfh.cv_results_)

In [ ]:
rf_best = rfh.best_estimator_
rf_best

In [ ]:
ytrainpred = rf_best.predict(xtrain)

### Model fit and prediction

### Evaluating model with test data<br>

The first baseline predictions are made on the test dataset.

In [110]:
train_final = d2wind.iloc[d2wind.index.get_level_values("date") >= pd.to_datetime("2021-07-12")].reset_index().set_index("date")

xtest = d_test2.reset_index().set_index("date").drop("total_amount", axis = 1)
ytest = d_test2.reset_index()[["date","total_amount"]].set_index("date")

In [111]:
xtest_final, ytestpred = pred_test(train = train_final, test = xtest, model = linear_model_transf)

S:\Repos\sweet-spot-donut-sales-forecasting\Function_scripts\functions_model.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:, "total_amount_lag_1"] = lag_value
S:\Repos\sweet-spot-donut-sales-forecasting\Function_scripts\functions_model.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:, "total_amount_lag_2"] = lag_value2
S:\Repos\sweet-spot-donut-sales-forecasting\Function_scripts\functions_model.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

### Fit statistics for train and test dataset

In [112]:
fit_overview(ytrain, ytrainpred, ytest, ytestpred)

R-squared train:  0.92
Mean absolute percentage error train:  26.0 

R-squared test  0.898
Mean absolute percentage error test:  26.0


For a baseline model, the final test results of the linear regression are relatively good. The **R-squared value of 0.9** is quite high for the test dataset. The **Mean Absolute Percentage Error (MAPE) of 26%** means that the model's predictions are, on average, 26% over or under the actual sales. The results can be further broken down into MAPE scores at the store and product category level.

In [90]:
mape_stores(d_test2, ytestpred, breakdown = "stores").sort_values("MAPE").reset_index(drop = True)

,Store name,MAPE
0,Jungfernstieg,[20.41255593230894]
1,Potsdamer,[21.340530140355074]
2,Neuer Kamp,[22.105932162432072]
3,Danziger,[23.036180544060255]
4,Mitte,[24.520883628633552]
5,Maybachufer,[25.94572683928529]
6,Warschauer,[30.493432624799645]
7,Altona,[37.26600070210256]


In [91]:
mape_stores(d_test2, ytestpred, breakdown = "stores_products").sort_values("MAPE").reset_index(drop = True)

,Store name,Product Category,MAPE
0,Altona,monthly_specials,[13.598857473097622]
1,Maybachufer,monthly_specials,[15.283161720350263]
2,Jungfernstieg,monthly_specials,[16.146499326368893]
3,Jungfernstieg,daily total,[16.395479279337188]
4,Mitte,monthly_specials,[16.457307949655057]
5,Potsdamer,monthly_specials,[18.038130538519273]
6,Neuer Kamp,monthly_specials,[18.280382458175662]
7,Potsdamer,classics,[18.368274261502446]
8,Maybachufer,daily total,[18.96270568608131]
9,Potsdamer,daily total,[19.29712567076772]


While some stores and store-product combinations were predicted relatively well and their MAPE scores are low, other stores and products don't exhibit such good results. Still, for a baseline model, the forecasted sales are solid, but can certainly be improved upon. This is especially true for those store-product combinations with a MAPE of over 30%.

Other models, especially tree-based methods, can be leveraged to improve upon the baseline model's predictions. This is further explored in the Random Forests and CatBoost notebooks.